<a href="https://colab.research.google.com/github/abidshafee/google.colaboratory_projects/blob/master/Price_Prediction_Model_using_SVM_and_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Program to predict asset price

In [9]:
# import quandl
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [13]:
# geting the eurousd quotes
df = web.DataReader('EURUSD=X', data_source='yahoo', start='2012-01-1', end='2020-7-14')
# view data
df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-07-07,1.134559,1.126519,1.127281,1.127523,0.0,1.127523
2020-07-08,1.137268,1.128796,1.133787,1.133915,0.0,1.133915
2020-07-09,1.132477,1.125720,1.128923,1.128796,0.0,1.128796
2020-07-12,1.137048,1.130454,1.130876,1.131004,0.0,1.131004
2020-07-14,1.140771,1.133016,1.134816,1.140251,0.0,1.140251


In [23]:
dataset = df[['Adj Close']]
dataset.head()

,Adj Close
Date,
2012-01-02,1.296008
2012-01-03,1.294096
2012-01-04,1.305602
2012-01-05,1.293142
2012-01-06,1.279607


In [30]:
# declaring a variable that holds the number days we want to predict
forcast = 2
# the adj. Close colum in dataset is our indipendent variable
# but we need a dependent variable that we will predict close price depends on adj. close price
# so now we will create another column in our dataset and will shift 'n' unit up
dataset['Predictions'] = dataset[['Adj Close']].shift(-forcast)
dataset.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Adj Close,Predictions
Date,,
2020-07-07,1.127523,1.128796
2020-07-08,1.133915,1.131004
2020-07-09,1.128796,1.140251
2020-07-12,1.131004,NaN
2020-07-14,1.140251,NaN


data in prediction colum is copied from data in adj close column but shifted two index. Because we want our model to predict the data in prediction column based on adj close column.

In [33]:
# Creaating independent dataset and converting it to numpy array
x_dataset = np.array(dataset.drop(['Predictions'], 1))
# now removing the last 'forcast' number of rows
x_dataset = x_dataset[:-forcast]
# independent dataset 'x_dataset' is a list of lists
x_dataset

array([[1.29600823],
       [1.29409635],
       [1.30560231],
       ...,
       [1.12752283],
       [1.13391542],
       [1.1287955 ]])

In [36]:
# Now creating dependent dataset and converting it also a numpy array including NaN
y_dataset = np.array(dataset['Predictions'])
# Getting all y_dataset values except the last NaN values
y_dataset = y_dataset[:-forcast]
# the dependent dataset y_dataset here is a list
y_dataset

array([1.30560231, 1.29314244, 1.27960682, ..., 1.1287955 , 1.13100421,
       1.14025092])

In [37]:
# Noow spliting the training data and testing data
x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, test_size = 0.2) 

## Creating Model-1 Using Support Vector Machine (SVM) Algorithm
as this is a regression problem therefore we will use SVR or support vector regression Algorithm

In [38]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

### Testing the SVR Model

In [42]:
svm_confidence = svr_rbf.score(x_test, y_test)
# best result is 1.0
print(svm_confidence)

0.6938676872993981


## Creating another model using linear regression

In [43]:
lr_model = LinearRegression()
# now training the lr_model
lr_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [45]:
# Testing Score of Linear Regrassion Model
lr_confidence = lr_model.score(x_test, y_test)
print(lr_confidence)

0.9923075110449259


This is really facinating confidence score for linear regression

In [50]:
# we will forcast the future data using the model
x_forcast = np.array(dataset.drop(['Predictions'], 1))[-forcast:]
print(x_forcast)

[[1.13100421]
 [1.14025092]]


Now we will predict the Adj Close price for the next 'forcast' nuber of days 

In [53]:
# prediction using Linear Regrassion
lr_predictions = lr_model.predict(x_forcast)
print(lr_predictions)

# prediction using SVM



[1.1311747  1.14037181]
